# Autor: Víctor Gullón Sánchez
# Fecha: 05/03/2024
# Tarea: 4 
# Asignatura: Sistemas Multimedia

## Parte 1

### 1.2: Graficas de audio
Para empezar tendremos que importar las librerias necesarias:

In [ ]:
pip install scipy

In [ ]:
# Importacion.
# import librosa
from scipy.io import wavfile
import IPython
import os
import numpy as np

In [ ]:
pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

Una vez tenemos las librerias importadas empezamos:
#### Audio estéreo:

In [ ]:
# Cargamos el archivo de audio.
filename = os.path.join("audio", 'breaking_bad.wav')
sample_rate, audio_data = wavfile.read(filename)

In [ ]:
# Se muestra la información del audio estéreo, donde se ve el canal1, canal2, aparte la resolución y el tamaño
print('Datos de audio (estereo):')
print(f'- Tamaño:     {audio_data.shape}')
print(f'- 1º canal:   {audio_data[:5, 0]}...')
print(f'- 2º canal:   {audio_data[:5, 1]}...')
print(f'- Resolucion: {type(audio_data[0,0])}\n')

print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz\n')

* Grafica de dominio:

In [ ]:
plt.subplot(2, 1, 1)  # Divide el espacio de la gráfica en 2 filas y 1 columna y selecciona la primera área de gráfica.
if audio_data.ndim > 1:  # Si 'audio_data' tiene más de una dimensión, lo que implica que puede ser un audio estéreo.
    for channel in range(audio_data.shape[1]):  # Itera sobre cada canal del audio estéreo (generalmente 2 canales).
        plt.plot(audio_data[:50, channel], label=f'Canal {channel + 1}')  # Grafica la onda de cada canal por separado.
    plt.title('Señal de Audio Estéreo')  # Pone un título a la gráfica.
else:
    plt.plot(audio_data)  # Si solo hay un canal (audio mono), grafica ese único canal.
    plt.title('Señal de Audio Estéreo (Canal Único)')  # Pone un título a la gráfica, aunque es mono se titula como estéreo por error.
plt.xlabel('Índice de Muestra')  # Etiqueta el eje x con 'Índice de Muestra'.
plt.ylabel('Amplitud')  # Etiqueta el eje y con 'Amplitud'.
plt.legend()  # Muestra una leyenda que identifica cada línea graficada.

#### Audio mono:

In [ ]:
# Se convierte el audio estéreo en mono, se entiende como una simplicación ya que de dos canales pasaremos a uno solo
new_data_mono = audio_data.mean(axis=1) # Se hace la media entre los valores de un canal y otro, y surge un solo canal
print('Nuevos datos de audio (mono):')
print(f'- Nuevo tamaño: {new_data_mono.shape}')
print(f'- Canal unico:  {new_data_mono[:5]}...')

# Mantenemos la misma resolucion que antes ya que esto no tiene por qué cambiar
new_data_mono = new_data_mono.astype(np.int16) # Se fuerza a que sea en ese tipo
print(f'- Resolucion:   {type(new_data_mono[0])}\n')

print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz\n')

* Grafica de dominio:

In [ ]:
sr_mono = 44100  # Tasa de muestreo de tu audio mono, ajusta según sea necesario.

# Calculando el tiempo en segundos para el eje x.
t_mono = np.arange(len(new_data_mono)) / sr_mono

# Creando la figura y el eje para la gráfica.
plt.figure(figsize=(10, 4))
plt.plot(t_mono, new_data_mono)
plt.title('Forma de Onda - Audio Mono: new_data_mono')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.show()

### 1.3: Definición de frecuencia de muestreo, aliasing, profundidad de
bits, ancho de banda y tasa de bits

* Frecuencia de muestreo: Número de veces por segundo que se toma una muestra de una señal analógica para convertirla en digital, determinando cuán bien se captura esta señal en el dominio digital.

* Aliasing: Distorsión que ocurre cuando la señal digitalizada representa incorrectamente las altas frecuencias de una señal analógica debido a una frecuencia de muestreo insuficiente.

* Profundidad de bits: Número de bits usados para representar cada muestra de una señal digital, afectando la precisión y el rango dinámico de la señal.

* Ancho de banda: Rango de frecuencias que puede transmitir o procesar un sistema, indicando la cantidad de información que puede manejar.

* Tasa de bits: Cantidad de datos transmitidos por segundo en una señal digital, influyendo en la calidad y el tamaño del archivo digital.

### 1.4: Audio mono, con la aplicación de la Transformada de Fourier para cambiar el dominio de la frecuencia. Se mostrará la gráfica y su explicación:

In [ ]:
n = len(new_data_mono)
Fs = sample_rate

# Calculando la Transformada Rapida de Fourier (FFT) en audio mono.
ch_Fourier = np.fft.fft(new_data_mono)  # ch1

# Solo miramos frecuencia por debajo de Fs/2
# (Nyquist-Shannon) --> Spectrum.
abs_ch_Fourier = np.absolute(ch_Fourier[:n//2])

# Graficamos.
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud', labelpad=10)
plt.xlabel('$f$ (Hz)', labelpad=10)
plt.show()

La Transformada de Fourier convierte un audio, que normalmente ves como una onda que sube y baja con el tiempo, en un montón de barras que muestran qué notas o tonos contiene ese audio y qué tan fuertes son.

### 1.5: Cálculo de la energía del espectrograma y la frecuencia de corte con un epsilon concreto

In [ ]:
# Definición del epsilon
eps = [ .155, .0, .1600]
eps = eps[2]
print(f'Epsilon: {eps}')

# Calculamos el valor de corte para esta energia.
thr_spec_energy = (1 - eps) * np.sum(abs_ch_Fourier)
print(f'Valor de corte para la energia del espectro: {thr_spec_energy}')

# Integral de la frecuencia --> energia del espectro.
spec_energy = np.cumsum(abs_ch_Fourier)

# Mascara (array booleano) que compara el
# valor de corte con la energia del espectro.
frequencies_to_remove = thr_spec_energy < spec_energy  
print(f'Mascara: {frequencies_to_remove}')

# La frecuencia f0 por la que cortamos el espectro.
f0 = (len(frequencies_to_remove) - np.sum(frequencies_to_remove)) * (Fs/2) / (n//2)
print(f'Frecuencia de corte f0 (Hz): {int(f0)}')

# Se muestra la gráfica
plt.axvline(f0, color='r')
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud')
plt.xlabel('$f$ (Hz)')
plt.show()

La definición del epsilon es el corte que vamos a realizar del audio para descartar, en este caso yo he escogido el 1600 para quedarme con la parte del audio donde las frecuencias son más altas, y descarto (de la línea roja hacia la derecha) las frecuencias más bajas o de menos energía.

### 1.6: En este punto vamos a comprimir la onda utilizando downsampling y obteniendo el factor a partir de la frecuencia de corte que se calculó previamente:

In [ ]:
# Definimos los nombres de los audios comprimidos.
wav_compressed_file = "AudioComprimido.wav"

# Calculamos el factor D de downsampling.
D = int(Fs / f0)
print(f'Factor de downsampling: {D}')

# Obtenemos los nuevos datos (slicing with stride).
new_data = new_data_mono[::D]

# Escribimos los datos a un archivo de tipo wav.
wavfile.write(
    filename=os.path.join('audio', wav_compressed_file),
    rate=int(Fs/D),
    data=new_data
)

# Cargamos el nuevo archivo.
sample_rate_comprimido, new_audio_data = wavfile.read(filename=os.path.join('audio', wav_compressed_file))

* slicing with stride: Esta función delimita que para la compresión del audio va a ir dando como saltos de elementos del array según el factor de downsampling, en este caso 7, que sale del cálculo con la frecuencia del audio original.

### 1.7: Espectograma de las ondas: original y comprimida. Diferencias:

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

Pxx, freqs, bins, im = ax[0].specgram(new_data_mono, NFFT=1024, Fs=sample_rate, noverlap=512)
ax[0].set_title('Espectograma del audio original')
ax[0].set_ylabel('Frecuencia (Hz)')
ax[0].grid(True)

Pxx, freqs, bins, im = ax[1].specgram(new_audio_data, NFFT=1024, Fs=sample_rate_comprimido, noverlap=512)
ax[1].set_title('Espectrograma del audio reducido/comprimido')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Frecuencia (Hz)')
ax[1].grid(True)

plt.tight_layout()
plt.show()

El espectrograma superior muestra un audio con un rango completo de frecuencias y detalles finos, mientras que el inferior tiene un rango de frecuencia limitado y menos detalles, indicando que se han eliminado las frecuencias altas y posiblemente se ha comprimido la señal, lo que resulta en una simplificación del contenido del audio.

### 1.8: Tamaño de los archivos:

* Tamaño del archivo comprimido:

In [ ]:
!ls -sh audio/AudioComprimido.wav

* Tamaño del archivo original:

In [ ]:
!ls -sh audio/breaking_bad.wav

### 1.9: Widgets para la reproducción del audio:

* Audio original:

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate)

* Audio comprimido:

In [ ]:
IPython.display.Audio(new_audio_data, rate=sample_rate_comprimido)